# Functions

The following snippet illustrates how to define a Python function:

In [1]:
def fact(n: int) -> int:
    """
    Return n! (n factorial)
    """
    result = 1
    while n > 0:
        result *= n
        n -= 1
    return result

You can obtain the value of the docstring using `fact.__doc__`. This is sometimes useful for identifying the function plotted on a chart, etc.

In [2]:
print(f"The function documented to {fact.__doc__} says {fact(5)=}")

The function documented to 
    Return n! (n factorial)
     says fact(5)=120


## Function parameter options

Python provide several ways to pass parameters to a function.

### Positional parameters

When passing parameters by position, the parameters used in the calling code are matched to the function's parameter variables based on their order.

In [3]:
def power(base, exponent):
    r = 1
    while exponent > 0:
        r *= base
        exponent -= 1
    return r

power(2, 3)

8

When using this method, the number of parameters used by the calling code exactly matches the number of parameters in the function definition, otherwise, a TypeError is raised.

In [5]:
try:
    power(3)
except Exception as e:
    print("Oops", {e})

Oops {TypeError("power() missing 1 required positional argument: 'exponent'")}


### Default values

Function parameters can have default values, which you declare by assigning a value in the parameter definition:

In [6]:
def power(base, exponent=2):
    r = 1
    while exponent > 0:
        r *= base
        exponent -= 1
    return r

assert power(2) == 4
assert power(2, 3) == 8

### Passing arguments by parameter name

Python allows you to pass arguments into a function by using the name of the corresponding function parameter. You don't need to do something special to enable this feature:

In [7]:
def power(base, exponent=2):
    r = 1
    while exponent > 0:
        r *= base
        exponent -= 1
    return r

assert power(exponent=2, base=3) == 9

This feature is called *keyword passing*. This technique, in combination with default arguments, is very useful when you're defining functions with a large numbers of possible arguments, most of which have common defaults.

For example, you could have a function such as:

```python
def list_file_info(show_size=False, show_creation_date=False, show_mod_date=False)
    ...
```

Even when the definition of the function is quite long and ugly, if a consumer only needs to retrieve the size of the file, the call will be:

```python
file_info = list_file_info(show_size=True)
```

### Variable number of arguments

Python allows functions to handle a variable number of arguments in two ways:
+ you can collect an unknown number of arguments at the end of the argument list into a list, commonly denoted as `*args`.
+ you can collect an arbitrary number of keyword arguments at the end of the argument list, and possibly after `*args` into a dictionary commonly denoted as `**kwargs`.

#### Dealing with an indefinite number of positional arguments

Prefixing the final parameter name of a function with a `*` causes all excess non-keyword arguments in a call of function to be collected together and assigned as a tuple to the given parameter.

In [3]:
def maximum(*numbers):
    if len(numbers) == 0:
        return None
    max_num = numbers[0]
    for n in numbers[1:]:
        if n > max_num:
            max_num = n
    return max_num

assert maximum(3, 2, 8) == 8
assert maximum(1, 5, 9, -2, -2, 9) == 9
assert maximum() == None
assert maximum(5) == 5

#### Dealing with an indefinite number of arguments passed by keyword

Prefixing the final parameter in the parameter list with `**` makes all the excess keyword-passed arguments to be collected as a dictionary.

The key for each entry in the dictionary will be parameter name.

In [7]:
def fun(x, y, **other):
    print(f"x: {x}, y: {y}, keys in other: {list(other.keys())}")
    for k in other.keys():
        print(f"other[{k!r}]={other[k]}")

fun(2, y=1, foo=3, bar="c")

x: 2, y: 1, keys in other: ['foo', 'bar']
other['foo']=3
other['bar']=c


### Mixing argument-passing techniques

It's possible to use all of the argument-passing techniques, provided that you follow this general rule:
1. Positional arguments come first.
2. Named arguments come afterwards.
3. An indefinite number of positional arguments identified by single parameter denoted by `*` come next.
4. An indefinite number of keyword arguments identified by a single argument denote by `**` come last.

```python
def fun(num1, str2, bool3, *args, **kwargs):
    ...

fun(2, "3", bool3=False, "catorce", 15, num=55, show_val="No")
```

#### Exercise

Write a function that can take any number of unnamed arguments and print their value in reverse order.

In [8]:
def fun(*args):
    for pos_arg in reversed(args):
        print(pos_arg)

fun(1, 2, 3)

3
2
1


### Mutable objects as arguments

Arguments are passed in by object reference. That is, once in the function, the parameter becomes a new reference to the object.

For immutable objects (such as tuples, strings, and numbers), what is done with a parameter has no effect outside the function.

To sum up:
+ if you pass a mutable object (such as a list, a dictionary, or a class instance), any change made to the object within the function will have an effect outside the function.
+ if you pass an immutable object (such as a tuple, a string, or a number), any change to the object will have no effect outside the function.

In [9]:
def f(n: int, list1: list, list2: list):
    list1.append(5)
    list2 = [4, 5, 6]
    n = n + 1

x = 5
y = [1, 2, 3, 4]
z = [5, 6]
f(x, y, z)

assert x == 5
assert y == [1, 2, 3, 4, 5]
assert z == [5, 6]

Note that:
+ `x` isn't changed because it is immutable and therefore, any change made to `x` will have no effect outside the function.
+ `y` is changed because it is mutable. As a result, appending an element through the reference `list1` will change the underlying list referenced by `y`.
+ `z` isn't changed although it is a mutable object because within the function, the parameter is reassigned to a new list `[4, 5, 6]`. Therefore, the underlying list originally received and referenced by `z` is unnchanged.

By contrast, the following code will change `z`:

In [11]:
def f(n: int, list1: list, list2: list):
    list1.append(5)
    list2[0] = 4
    list2[1] = 5
    list2.append(6)
    n = n + 1

x = 5
y = [1, 2, 3, 4]
z = [5, 6]
f(x, y, z)

assert z == [4, 5,6]

### Mutable objects as default values

Passing mutable objects as parameter values can cause bugs, but it is quite often the most convenient and efficient way to do things &mdash; you just need to be aware that changing the object might have an effect outside the function.

It's much worse to use a mutable object as a default parameter value and then mutate the object.

Let's see that with the following function that receives a list of numbers as the first parameter and a second list in which the odd numbers from the first list will be collected:

In [14]:
def odd_numbers(test_list, odds):
    for n in test_list:
        if n % 2:
            odds.append(n)

odds = []
odd_numbers([1, 5, 7, 9, 10], odds)

print(f"{odds}")

[1, 5, 7, 9]


Let's now make the function a bit weirder and also return `odds`. Also, to give the function a better DX you decide to initialize `odds` with an empty list, so that the user does not need to supply one:

In [2]:
def odd_numbers(test_list, odds=[]):
    for n in test_list:
        if n % 2:
            odds.append(n)
    return odds


odds = odd_numbers([1, 5, 7, 9, 10])

print(f"{odds}")

[1, 5, 7, 9]


At first sight, it seems that the function works as expected, but if we call the function again:

In [3]:
odd_numbers([1, 5, 7, 9, 10], odds)

print(f"{odds}")

[1, 5, 7, 9, 1, 5, 7, 9]


See that the `odds` parameter still remembers the state from the previous execution, which is not what we intended!

This happens because when a default parameter is used, Python assigns the object to be used as the default value when the function is first compiled, and it does not change it for the entire duration of the program.

Therefore, when you have a mutable object as the default and mutate, every time that the default value is used, it will be the same object, and that object will reflect all of the times the function has been called with it.

| NOTE: |
| :---- |
| To minimize that problem is recommended to use `None` as the default value of mutable objects that you pass to functions. |

## Local, nonlocal, and global variables

Consider the following function, which implements the factorial using an iterative approach:

In [1]:
def fact(n:int) -> int:
    r = 1
    while n > 0:
        r *= n
        n -= 1
    return r

fact(5)

120

Both the variables `r` and `n` are local to any particular call of the factorial function.

This means that changes to them made when the function is executing have no effect on any variables outside the function.

In Python, global variables are declared and accessed using a different approach than in other programming languages. You can make a variable global by declaring it so, before the variable is used, using the `global` statement.

The following snippet declares `a` as a global variable so that it can be accessed and changed by the function:

In [4]:
a = 5
b = 55

def func():
    global a
    print(f"a={a}")
    a = 1
    b = 2

func()

assert a == 1
assert b == 55


a=5


See how the value of `a` is changed, while the value of `b` (being local to the funcion) is not altered outside the function.

Note that global variables that you just want to access and not changed them do not require the `global` statement:

In [5]:
b = 55

def func2():
    print(f"b={b}")

func2()

b=55


However, if you try to modify `b` within the `func2` body, Python will create a new local variable instead, which will result in an exception because you're trying to use a variable without having given it a value first:

In [9]:
b = 55

def func2():
    print(f"b={b}")
    b = 5
    print(f"b={b}")

try:
    func2()
except Exception as e:
    print(f"Oops:", {e})


Oops: {UnboundLocalError("cannot access local variable 'b' where it is not associated with a value")}


The `nonlocal` statement causes an identifier to refer to a previously bound variable in the closest enclosing scope. It is therefore similar to `global`, but to refer to variables defined in an enclosing scope such as an outer function:

In [15]:
g_var = 0
nl_var = 0
print(f"top level-> g_var: {g_var}, nl_var: {nl_var}")

def outer_fn():
    nl_var = 2  # hides nl_var defined in the global scope
    print(f"\tin outer_fn-> g_var: {g_var}, nl_var: {nl_var}")
    assert g_var == 0
    assert nl_var == 2
    def inner_fn():
        global g_var
        nonlocal nl_var
        g_var = 1
        nl_var = 4
        print(f"\t\tin inner_fn-> g_var: {g_var}, nl_var: {nl_var}")
        assert g_var == 1
        assert nl_var == 4

    inner_fn()
    print(f"\tin outer_fn-> g_var: {g_var}, nl_var: {nl_var}")
    assert g_var == 1
    assert nl_var == 4

outer_fn()
print(f"top level-> g_var: {g_var}, nl_var: {nl_var}")
assert g_var == 1
assert nl_var == 0

top level-> g_var: 0, nl_var: 0
	in outer_fn-> g_var: 0, nl_var: 2
		in inner_fn-> g_var: 1, nl_var: 4
	in outer_fn-> g_var: 1, nl_var: 4
top level-> g_var: 1, nl_var: 0


Therefore in Python:
> if you want to assign to a variable existing outside a function, you must explicitly declare the variable to be `nonlocal` or `global`.<br>If you're just reading the variable, you don't need to declare it.

### Exercise

Assuming that `x = 5`, what will be the value of `x` after `funct1()` is executed? And after `funct2()` is executed?

```python
def funct1():
    x = 3

def funct2():
    global x
    x = 2
```

We can use `assert` to check our expectations:

In [16]:
x = 5

def funct1():
    x = 3

def funct2():
    global x
    x = 2

funct1()
assert x == 5

funct2()
assert x == 2

## Assigning functions to variables

In Python, functions are first-class citizens. As such, functions can be assigned to variables just like numbers, lists, or tuples:

In [18]:
def f_to_kelvin(degrees_f):
    return 273.15 + (degrees_f - 32) * 5 / 9

def c_to_kelvin(degrees_c):
    return 273.15 + degrees_c

abs_temp = f_to_kelvin
print(f"{abs_temp(32)=}")

abs_temp = c_to_kelvin
print(f"{abs_temp(0)=}")


abs_temp(32)=273.15
abs_temp(0)=273.15


Note how a variable that refers to a function can be used in the exact same way as the function itself.

You can place functions in lists, tuples, or dictionaries, as you'd do with other variables:

In [19]:
t_conversors = {
    "f_to_k": f_to_kelvin,
    "c_to_k": c_to_kelvin
}

t_conversors["c_to_k"](0)

273.15

## Lambda expressions

Short functions like the ones from the previous section, can be defined using the lambda expression syntax:

```python
lambda param1, param2, ..., paramN: expression
```

This syntax is appropriate for small functions that need to be defined inline and passed to another function, or kept in some other object or variable:

In [20]:
t_conversors = {
    "f_to_k": lambda t_f : 273.15 + (t_f - 32) * 5 / 9,
    "c_to_k": lambda t_c : 273.15 + t_c
}

print(f"{t_conversors["f_to_k"](32)=}")
print(f"{t_conversors["c_to_k"](0)=}")

t_conversors["f_to_k"](32)=273.15
t_conversors["c_to_k"](0)=273.15


Note that lambda expressions don't have a `return` statements because the value of the expression is automatically returned.

## Generator functions

A **generator** function is a special kind of function that lets you define your own iterators.

When using a generator, you use `yield` keyword to return each iteration's value.

When a generator function is used, it returns a generator object, which can be used as an iterator. As such, the generator function body will execute up to the `yield` statement at which point it will return a value and the execution of the generator function will stop.

When invoked again, the generator function execution will resume after `yield` and will execute until either `yield` is found again (in which case the execution will temporarily stop again), or until it finds an empty `return` statement which will mean that the generator function has finished its execution.

In [21]:
def four():
    x = 0
    while x < 4:
        print("\t>> in generator, x =", x)
        yield x
        x += 1

for i in four():
    print(f"Using value from the generator: {i}")

	>> in generator, x = 0
Using value from the generator: 0
	>> in generator, x = 1
Using value from the generator: 1
	>> in generator, x = 2
Using value from the generator: 2
	>> in generator, x = 3
Using value from the generator: 3


Starting with Python 3.3, a new keyword `yield from` is also available.

The keyword `yield from` lets you chain generators together:

In [22]:
def inner_gen(x):
    for i in range(x):
        yield i

def outer_gen(y):
    yield from inner_gen(y)

for i in range(6):
    print(i)

0
1
2
3
4
5


See how `yield from` behaves the same way `yield` does, except that it delegates the generator machinery to a subgenerator.

You can use generator functions with `in` to see whether a value is the the series that a generator produces:

In [29]:
def four():
    n = 0
    while n <= 4:
        print("\t>> yielding ", n)
        yield n
        n += 1

assert (2 in four()) == True
assert (5 in four()) == False

if 2 in four():
    print("2 will be yielded by four")

if 5 in four():
    print("5 will be yielded by four")
else:
    print("5 will be yielded by four")


	>> yielding  0
	>> yielding  1
	>> yielding  2
	>> yielding  0
	>> yielding  1
	>> yielding  2
	>> yielding  3
	>> yielding  4
	>> yielding  0
	>> yielding  1
	>> yielding  2
2 will be yielded by four
	>> yielding  0
	>> yielding  1
	>> yielding  2
	>> yielding  3
	>> yielding  4
5 will be yielded by four


Note that when using `in` the generator will be invoked until either the value is found or the generator is exhausted.

## Decorators

The fact that a function can be assigned to a variable, and that variable used to invoke the function it references can be exploited to write a Python function that takes another function as its parameter, wraps it in another function that does something related, and then returns the new function.

This new function can be used instead of the original function:

In [30]:
def decorate(fn: callable):
    print(f"\t>> in decorate function, decorating {fn.__name__}")
    def wrapper_fn(*args):
        print(f"\t\t>> executing {fn.__name__}")
        return fn(*args)
    return wrapper_fn

def my_fn(param):
    print(param)

my_fn(5)

my_decorated_fn = decorate(my_fn)
my_decorated_fn(5)

5
	>> in decorate function, decorating my_fn
		>> executing my_fn
5


Python defines the syntax `@decorate`, where `decorate` is a function like the one define above, to simplify the way in which functions can be decorated:

In [31]:
def decorate(fn: callable):
    print(f"\t>> in decorate function, decorating {fn.__name__}")
    def wrapper_fn(*args):
        print(f"\t\t>> executing {fn.__name__}")
        return fn(*args)
    return wrapper_fn

@decorate
def my_fn(param):
    print(param)

my_fn(55)

	>> in decorate function, decorating my_fn
		>> executing my_fn
55


Note that the effect of using the syntax `@decorate` is exactly the same as the code block:

```python
def my_fn(param):
    print(param)

my_decorated_fn = decorate(my_fn)
```

but much more succinct, and therefore, preferred.

### Exercise

Create a decorator function that enclose the return value of the wrapped function in `"<html>"` and `"</html>"`.

In [1]:
def html_tag(fn: callable):
    def wrapper_fn(*args):
        return f"<html>{fn(*args)}</html>"
    return wrapper_fn

@html_tag
def my_fn(param):
    return param

print(my_fn("Hello"))

<html>Hello</html>


### Exercise

Refactor using functions the exercise at the end of the [04: Dictionaries](../04_dictionaries/04_dictionaries.ipynb) chapter that cleans and counts the occurences of words in a text file.

In [5]:
import string

def get_normalized_words_from_line(line):
    """
    Return the list of normalized words from the line.

    Args:
        line: the input line to normalize

    Returns:
        a list of words that have been normalized to facilitate
        word frequency analysis.
    """
    line_lowercase = line.lower()
    normalized_line = line_lowercase.translate(str.maketrans("", "", string.punctuation))
    return normalized_line.split()

def write_normalized_words_to_file(input_file):
    """
    Write the normalized words from the input file to a new file.

    Args:
        input_file: the file to read the input from
    """
    with open(input_file, "r") as infile:
        with open("data/out/normalized_words.txt", "w") as outfile:
            for line in infile:
                words = get_normalized_words_from_line(line)
                for word in words:
                    outfile.write(f"{word}\n")

def get_word_occurrences_from_file(input_file):
    """
    Return the dictionary of word occurrences from the input file.

    Args:
        input_file: the file to read the input from

    Returns:
        a dictionary of word occurrences
    """
    word_occurrences = {}
    with open(input_file, "r") as infile:
        for word in infile:
            word = word.strip()
            word_occurrences[word] = word_occurrences.get(word, 0) + 1
    return word_occurrences

def print_word_occurrences(word_occurrences):
    for word, count in word_occurrences.items():
        print(f"{word}: {count}")

def get_top_n_most_frequent_words(word_occurrences, n):
    return sorted(word_occurrences.items(), key=lambda x: x[1], reverse=True)[:n]

def get_top_n_least_frequent_words(word_occurrences, n):
    return sorted(word_occurrences.items(), key=lambda x: x[1])[:n]

write_normalized_words_to_file("data/moby_01.txt")
word_occurrences = get_word_occurrences_from_file("data/out/normalized_words.txt")
print_word_occurrences(word_occurrences)
print("Top 10 most frequent words:", get_top_n_most_frequent_words(word_occurrences, 10))
print("Top 10 least frequent words:", get_top_n_least_frequent_words(word_occurrences, 10))

call: 1
me: 5
ishmael: 1
some: 2
years: 1
ago: 1
never: 1
mind: 1
how: 1
long: 1
precisely: 1
having: 1
little: 2
or: 2
no: 1
money: 1
in: 4
my: 4
purse: 1
and: 9
nothing: 2
particular: 1
to: 5
interest: 1
on: 1
shore: 1
i: 9
thought: 1
would: 1
sail: 1
about: 2
a: 6
see: 1
the: 14
watery: 1
part: 1
of: 8
world: 1
it: 6
is: 7
way: 1
have: 1
driving: 1
off: 2
spleen: 1
regulating: 1
circulation: 1
whenever: 4
find: 2
myself: 2
growing: 1
grim: 1
mouth: 1
damp: 1
drizzly: 1
november: 1
soul: 1
involuntarily: 1
pausing: 1
before: 1
coffin: 1
warehouses: 1
bringing: 1
up: 1
rear: 1
every: 1
funeral: 1
meet: 1
especially: 1
hypos: 1
get: 2
such: 1
an: 1
upper: 1
hand: 1
that: 2
requires: 1
strong: 1
moral: 1
principle: 1
prevent: 1
from: 1
deliberately: 1
stepping: 1
into: 1
street: 1
methodically: 1
knocking: 1
peoples: 1
hats: 1
then: 1
account: 1
high: 1
time: 2
sea: 1
as: 3
soon: 1
can: 1
this: 2
substitute: 1
for: 1
pistol: 1
ball: 1
with: 3
philosophical: 1
flourish: 1
cato: 1
throws: